In [ ]:
# This cell tells python to use the local version of CVToolkit instead of the installed one.
# Use for testing purposes until branch feature/BCV-970-oor-metrics is merged.
import sys
import os

module_path = os.path.abspath(os.path.join("../../CVToolkit"))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
import sys
import pandas as pd

sys.path.append("../..")

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.oor_evaluation import (
    tba_evaluation, coco_evaluation
)
from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.metrics_utils import (
    BoxSize, ObjectClass, predictions_to_coco_json
)

In [ ]:
# Input: ground truth and predictions
gt_annotations_folder = "../../datasets/oor/processed-first-official-training-dataset-oor/labels/val"
gt_coco_json = "../../datasets/oor/processed-first-official-training-dataset-oor/coco_annotations_val.json"

# Size of the images (width, height)
img_shape = (3840, 2160)

# models = ["yolov8s_1024", "yolov8m_1024", "yolov8s_1280", "yolov8m_1280", "yolov8m_1920"]
models = ["yolov8m_1280", "yolov8m_1920"]

In [ ]:
from cvtoolkit.datasets.yolo_labels_dataset import YoloLabelsDataset

ds1 = YoloLabelsDataset(
    folder_path=gt_annotations_folder,
    image_area=img_shape[0]*img_shape[1],
)
ds2 = YoloLabelsDataset.from_yolo_validation_json(
    yolo_val_json=gt_coco_json,
    image_shape=img_shape,
)

In [ ]:
# Compute TBA results

tba_results = []

for model in models:
    pred_folder = f"../../experiments/pred_val_{model}/labels"
    # pred_folder = f"../../experiments/pred_val_{model}/coco_predictions.json"
    tba_results_file = f"../../experiments/pred_val_{model}/tba_results.md"

    tba_results.append(
        tba_evaluation(
            ground_truth_folder=gt_annotations_folder,
            # ground_truth_folder=gt_coco_json,
            prediction_folder=pred_folder,
            image_shape=img_shape,
            save_results=False,
            results_file=tba_results_file,
            hide_progress=True,
            upper_half=False,
        )
    )

In [ ]:
from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics.per_pixel_stats import EvaluatePixelWise

EvaluatePixelWise.store_tba_results(tba_results, model_name=models, markdown_output_path="tba_results_old.md")

In [ ]:
# Compute per pixel results 

if False:
    per_pixel_results = tba_evaluation(
        ground_truth_folder=gt_annotations_folder,
        prediction_folder=pred_folder,
        image_shape=img_shape,
        object_classes=ObjectClass,
        single_size_only=True,
        save_results=False,
    )

    pd.DataFrame(data=per_pixel_results).transpose()

In [ ]:
# Compute COCO evaluation

data_labels = [
    "model_size",
    "img_size",
    "target_class",
    "AP@50-95_all",
    "AP@75_all",
    "AP@50_all",
    "AP@50_small",
    "AP@50_medium",
    "AP@50_large",
    "AR@50-95_all",
    "AR@75_all",
    "AR@50_all",
    "AR@50_small",
    "AR@50_medium",
    "AR@50_large",
]

target_classes = [[0, 1, 2], [0], [1], [2]]
target_class_names = ["all", "person", "license_plate", "container"]

coco_df = pd.DataFrame(columns=data_labels)

for model in models:
    pred_folder = f"../../experiments/pred_val_{model}/labels"
    pred_coco_json = f"../../experiments/pred_val_{model}/coco_predictions.json"

    # if not os.path.isfile(pred_coco_json):
    predictions_to_coco_json(predictions_folder=pred_folder, image_shape=img_shape, json_file=pred_coco_json)

    model_size = model.split(sep="_")[0][-1]
    img_size = int(model.split(sep="_")[1])

    for target_cls_name, target_cls in zip(target_class_names, target_classes):
        print(f"EVALUATING {model}, TARGET CLASS {target_cls_name}")
        eval = coco_evaluation(
            coco_annotations_json=gt_coco_json,
            coco_predictions_json=pred_coco_json,
            predicted_img_shape=img_shape,
            class_ids=target_cls,
            print_summary=True,
        )
        coco_df.loc[f"{model}_{target_cls_name}"] = [model_size, img_size, target_cls_name, *eval.stats]

In [ ]:
coco_df.to_csv("coco_results.csv")

In [ ]:
# For demo
pd.set_option('display.float_format', '{:.2f}'.format)

cols_to_show = [
    "target_class",
    "AP@50_all",
    "AP@50_small",
    "AP@50_medium",
    "AP@50_large",
    "AR@50_all",
    "AR@50_small",
    "AR@50_medium",
    "AR@50_large",
]

demo_df = coco_df[(coco_df["model_size"]=="m") & (coco_df["img_size"].isin((1024, 1920)))]
demo_df = demo_df[cols_to_show]

In [ ]:
demo_df[demo_df["target_class"]=="all"]

In [ ]:
demo_df[demo_df["target_class"]=="container"]